# The Spinning Effective One-Body Hamiltonian: "v5HM" (For Calibration)

## Author: Siddharth Mahesh

## This module documents the reduced spinning effective one-body Hamiltonian for the sake of calibration tests.


**Notebook Status:** <font color='green'><b> Validation </b></font>

**Validation Notes:** This module has been validated against the [pySEOBNR code](https://git.ligo.org/waveforms/software/pyseobnr/) that was reviewed and approved for LIGO parameter estimation by the LIGO Scientific Collaboration.  That is, the value $H_{\rm real}$ output from this notebook agrees to roundoff error with the value of $H_{\rm real}$ computed by pyseobnr for SEOBNRvHM's calibration fits.

### NRPy+ Source Code for this module: [v5HM_unoptimized_hamiltonian.py](./Hamiltonian/v5HM_unoptimized_hamiltonian.py)

<a id='intro'></a>

## Introduction
$$\label{intro}$$

### The Physical System of Interest

Consider two black holes with masses $m_{1}$, $m_{2}$ and spins ${S}_{1}$, ${S}_{2}$ that are aligned (i.e parallel or antiparallel with respect to each other) in a binary system.  The spinning effective one-body ("SEOB") Hamiltonian $H_{\rm real}$ (defined in [this cell](#hreal)) describes the conservative dynamics of this system; we will define $H_{\rm real}$ as in [Pompili, Buonnano et. al](https://arxiv.org/pdf/2303.18039) Appendix A \& B.  There, $H_{\rm real}$ is canonically transformed and mapped to an effective Hamiltonian $H_{\rm eff}$ (defined in [this cell](#heff)) describing the motion of a test particle of mass $\mu$ (defined in [this cell](#mu)) moving at the equator of a deformed Kerr metric. Here we seek to break up $H_{\rm real}$ and document the terms in such a way that the resulting Python code can be used to numerically evaluate $H_{\rm real}$.

We write $H_{\rm real}$ in terms of $r$, and $\phi, the [Boyer-Lindquist coordinates](https://en.wikipedia.org/wiki/Boyer%E2%80%93Lindquist_coordinates) (see [Barausse and Buonanno (2010)](https://arxiv.org/abs/0912.3517) Section IV).

Please note that throughout this notebook we adopt the following convention $G = M = c = 1$, where $G$ is the universal gravitational constant, $M = m_1 + m_2$ is the total mass of the binary and, $c$ is the speed of light in vacuum.

Running this notebook to completion will generate a file called v5HM_unoptimized_hamiltonian.py. This file contains the Python function v5HM_unoptimized_hamiltonian(), which takes as input m1, m2 (each in solar masses), the values of the normalized spins $\chi_{1,2} = \frac{S_{1,2}}{m_{1,2}^2}$ and values for all four dynamic variables (the radial separation $r$, the azimuthal angle $\phi$, the radial tortoise momentum $p_{r_{*}}$ and, the angular momentum $p_{\phi}$).

### Citations
Throughout this module, we will refer to
* [Pompili, Buonanno, et al (2023)](https://arxiv.org/pdf/2303.18039.pdf) as PB2023,
* [SEOBNRv5HM-Notes](https://dcc.ligo.org/public/0186/T2300060/002/SEOBNRv5HM.pdf) as V5HM,
* [pySEOBNR documentation/code](https://git.ligo.org/waveforms/software/pyseobnr/) as pySEOBNR.

<a id='outputcreation'></a>

# Step 0: Creating the output directory for SEOBNR \[Back to [top](#toc)\]
$$\label{outputcreation}$$

First we create the output directory for SEOBNR (if it does not already exist):

In [1]:
import sys#Add sys to get cmdline_helper from NRPy top directory; remove this line and next when debugged
sys.path.append('../')
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

# Create C code output directory:
Ccodesdir = "Hamiltonian"
# Then create an output directory in case it does not exist
cmd.mkdir(Ccodesdir)

<a id='hreal'></a>

# Step : The real Hamiltonian $H_{\textrm{real}}$ \[Back to [top](#toc)\]
$$\label{hreal}$$

The SEOB Hamiltonian $H_{\rm real}$ is given by [RBB2023](https://arxiv.org/abs/2303.18046) Equation (8):

\begin{equation*}
    H_{\rm real} = \sqrt{ 1 + 2 \nu \left( \frac{ H_{\rm eff} }{ \mu } - 1 \right) }.
\end{equation*}

Here $H_{\rm eff}$ (defined in [this cell](#heff)) is an *effective* Hamiltonian (see [this cell](#intro)), and $\nu$ (defined in [this cell](#nu)) is the symmetric mass ratio.

In [2]:
%%writefile $Ccodesdir/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt
Hreal = sp.sqrt(1 + 2*nu*(Heff - 1))

Writing Hamiltonian/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt


<a id='heff'></a>

# Step : The effective Hamiltonian $H_{\textrm{eff}}$ \[Back to [top](#toc)\]
$$\label{heff}$$

The effective Hamiltonian $H_{\rm eff}$ as stated in [RBB2023](https://arxiv.org/abs/2303.18046) under equation B2, is split into odd and even-in-spin components $H_{\rm odd}$, and $H_{\rm even}$ but expressed in Equation A:

\begin{equation*}
    H_{\rm eff} = \underbrace{ \frac{ p_{\phi}\left( g_{a_{+}} a_{+} + g_{a_{-}} \delta a_{-} \right) + {\rm SO}_{\rm calib} + G^{\rm align}_{a^3}}{r^3 + a_{+}^2\left(r + 2\right)} }_{H_{\rm odd}} + \underbrace{ \sqrt{A^{\rm align} \left( 1 + \frac{p_{\phi}^{2}}{r^2} + \left(1 + B_{np}^{align}\right)({\bf n}\cdot{\bf p})^2 + B_{npa}^{\rm Kerr eq} \frac{p_{\phi}^{2}\left({\bf l}\cdot{\bf a}_{+}\right)^2}{r^2} + Q^{\rm align} \right) } }_{H_{\rm even}}.
\end{equation*}

Here, $H_{\rm even}$ is defined in [this cell](#heven), and $H_{\rm odd}$ is defined in [this cell](#hodd). 

In [3]:
%%writefile -a $Ccodesdir/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt
Heff = Hodd + Heven

Appending to Hamiltonian/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt


<a id='hodd'></a>

# Step : The odd-in-spin Hamiltonian $H_{\textrm{odd}}$ \[Back to [top](#toc)\]
$$\label{hodd}$$

The effective Hamiltonian $H_{\rm eff}$ as stated in [RBB2023](https://arxiv.org/abs/2303.18046) under equation B2, is split into odd and even-in-spin components $H_{\rm odd}$, and $H_{\rm even}$. Here, we state the odd component in Equation A:

\begin{equation*}
    H_{\rm odd} = \frac{ Mp_{\phi}\left( g_{a_{+}} a_{+} + g_{a_{-}} \delta a_{-} \right) + {\rm SO}_{\rm calib} + G^{\rm align}_{a^3}}{r^3 + a_{+}^2\left(r + 2*M\right)}.
\end{equation*}

Here, $M$, $p_{\phi}$, $r$ are the total mass, azimuthal momentum and radial distance respectfully. ${\bf l}\cdot{\bf a}_{\pm}$ is defined in [this cell](#la), the gyromagnetic constants $g_{a_{\pm}}$ are defined in [this cell](#ga) the magnitude of ${\bf a}_{+}$ is defined in [this cell](#a),  ${\rm SO}_{\rm calib}$ is defined in [this cell](#socalib), and $\langle G^{\rm pprec}_{a^3} \rangle$ is defined in [this cell](#gppreca3). 

In [4]:
%%writefile -a $Ccodesdir/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt
Hodd = (pphi*(gap*ap + delta*gam*am) + SOcalib + Galigna3)/(r**3 + ap**2*(r + 2))

Appending to Hamiltonian/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt


<a id='heven'></a>

# Step : The even-in-spin Hamiltonian $H_{\textrm{even}}$ \[Back to [top](#toc)\]
$$\label{heven}$$

The effective Hamiltonian $H_{\rm eff}$ as stated in [RBB2023](https://arxiv.org/abs/2303.18046) under equation B2, is split into odd and even-in-spin components $H_{\rm odd}$, and $H_{\rm even}$. Here, we state the even component in Equation A:

\begin{equation*}
    H_{\rm even} = \sqrt{A^{\rm align} \left( 1 + \frac{p_{\phi}^{2}}{r^2} + \left(1 + B_{np}^{\rm align}\right)({\bf n}\cdot{\bf p})^2 + B_{npa}^{\rm Kerr eq} \frac{p_{\phi}^{2}\left({\bf l}\cdot{\bf a}_{+}\right)^2}{r^2} + Q^{\rm align} \right) }.
\end{equation*}

Here, $r$, and $p_\phi$ are the radius, and the azimuthal momentum respectively and the radial momentum $p_{r}$ is defined in [this cell](#pr). $A^{\rm align}$ is defined in [this cell](#aalign), $B_{np}^{\rm align}$ is defined in [this cell](#bnpalign), $B_{npa}^{\rm Kerr eq}$ is defined in [this cell](#bnpakerreq), ${\bf l}\cdot{\bf a}_{+}$ is defined in [this cell](#lap), and $Q^{\rm align}$ is defined in [this cell](#qalign). 

In [5]:
%%writefile -a $Ccodesdir/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt
Heven = sp.sqrt(
    Aalign * (
        1 
        + pphi*pphi/r**2 
        + (1 + Balignnp)*pr*pr 
        + Bkerreqnp*pphi*pphi*ap**2/r**2 
        + Qalign
    )
)

Appending to Hamiltonian/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt


# Step: The Odd-In-Spin Terms

<a id='socalib'></a>

# Step : ${\textrm{SO}}_{\textrm{calib}}$ \[Back to [top](#toc)\]
$$\label{socalib}$$

The SO calibration term is expressed in [RBB2023](https://arxiv.org/abs/2303.18046) Equation A2. The calibration coefficient $d_{\rm SO}$ is chosen to be a free parameter that we will calibrate to NR simulations.:

\begin{equation*}
    {\rm SO}_{\rm calib} = \nu d_{\rm SO} \frac{p_{\phi}}{r^3}  {a}_{+}, \\
\end{equation*}

Here, $r$, and $p_\phi$ are the radial distance and the azimuthal momentum respectively. $\nu$, is the symmetric mass ratio defined in [this cell](#nu), ${\bf a}_{\pm}$ is defined in [this cell](#a), ${\bf l}\cdot{\bf a}_{+}$ is defined in [this cell](#la).

In [6]:
%%writefile -a $Ccodesdir/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt
SOcalib = nu*dSO*ap*pphi*(u**3)

Appending to Hamiltonian/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt


<a id='galigna3'></a>

# Step : $G_{a^3}^{\textrm{align}}$ \[Back to [top](#toc)\]
$$\label{gpreca3}$$

The cubic-in-spin term $G_{a^3}^{\rm prec}$ is expressed in [LB2023](https://arxiv.org/abs/2303.18046) Equation A4:
\begin{equation*}
    G_{a^3}^{\rm prec} = \frac{p_{\phi}}{4r^2}\left( \delta a_{-}a_{+}^{2} - a_{+}^{3} \right) 
\end{equation*}

Here, $r$, and $p_\phi$ are the radial distance and the azimuthal momentum respectively.$\delta$ is defined in [this cell](#delta), and ${a}_{\pm}$ is defined in [this cell](#a).

In [7]:
%%writefile -a $Ccodesdir/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt
Galigna3 = pphi*(delta*am*ap**2 - ap**3)/(4*r**2)

Appending to Hamiltonian/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt


# Step : The Even-In-Spin Terms

<a id='aalign'></a>

# Step : $A^{\textrm{align}}$ \[Back to [top](#toc)\]
$$\label{apprec}$$

The potential $A^{\rm align}$ is expressed in [RBB2023](https://arxiv.org/abs/2303.18046) Equation A5:
\begin{equation*}
    A^{\rm align} = \frac{a_{+}^{2}/r^{2} + A_{\rm noS} + A^{align}_{\rm SS}}{1 + \left(1 + 2/r\right)a_{+}^{2}/r^{2}}.
\end{equation*}

Here, $r$ is the radial distance and $a_{+}$ is defined [in this cell](#a). The spin-spin contribution, $ A_{\rm SS}^{\rm align}$ is defined in [this cell](#aalignss), and the non-spinning contribution, $A_{\rm noS}$ is defined in [this cell](#anons).

In [8]:
%%writefile -a $Ccodesdir/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt
Aalign = (ap**2*u*u + Anons + AalignSS )/(1 + ap**2*(1 + 2/r)/(r**2) )

Appending to Hamiltonian/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt


<a id='bkerreqnpa'></a>

# Step : $B_{npa}^{\textrm{Kerr eq}}$ \[Back to [top](#toc)\]
$$\label{bkerrnpa}$$

The potential $B_{npa}^{\rm Kerr eq}$ is expressed in [RBB2023](https://arxiv.org/abs/2303.18046) Equation A4:
\begin{equation*}
    B_{npa}^{\rm calib} = -\frac{1 + 2/r}{r^2 + a_{+}^2\left(1 + 2/r\right)}.
\end{equation*}

Here, $r$ is the radial distance, and ${a}_{+}$ is defined in [this cell](#a).

In [9]:
%%writefile -a $Ccodesdir/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt
Bkerreqnp = - (1 + 2/r)/(r**2 + ap**2*(1 + 2/r))

Appending to Hamiltonian/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt


<a id='bnpalign'></a>

# Step : $B_{p}^{\textrm{align}}$ \[Back to [top](#toc)\]
$$\label{bnpalign}$$

The potential $B_{np}^{\rm align}$ is expressed in [RBB2023](https://arxiv.org/abs/2303.18046) Equation A5:
\begin{equation*}
    B_{np}^{\rm align} = -1 + a_{+}^{2}/r^{2} + A_{\rm noS}\bar{D}_{\rm noS} + {B}_{np,\rm SS}^{\rm align}.
\end{equation*}

Here, $r$ is the radial distance, and $a_{+}$ is defined in [this cell](#a). The spin-spin contribution $ B_{np,\rm SS}^{\rm prec}$ is defined in [this cell](#bnpssalign). The non-spinning potential $A_{\rm noS}$ is defined in [this cell](#anons), and $\bar{D}_{\rm noS}$ is defined in [this cell](#dnons)

In [10]:
%%writefile -a $Ccodesdir/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt
Balignnp = -1 + ap**2*u*u + Anons*Dnons + BnpalignSS

Appending to Hamiltonian/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt


<a id='qalign'></a>

# Step : $Q^{\textrm{align}}$ \[Back to [top](#toc)\]
$$\label{qalign}$$

The potential $Q^{\rm align}$ is expressed in [RBB2023](https://arxiv.org/abs/2303.18046) Equation A5:
\begin{equation*}
    Q^{\rm align} = Q_{\rm noS} + Q_{\rm SS}^{\rm align}.
\end{equation*}

The spin-spin contribution $ Q_{\rm SS}^{\rm align}$ is defined in [this cell](#qssalign). The non-spinning contribution, $Q_{\rm noS}$ is defined in [this cell](#qnos).

In [11]:
%%writefile -a $Ccodesdir/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt
Qalign = Qnos + QalignSS

Appending to Hamiltonian/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt


# Step : The Spin-Spin Contributions

<a id='aalignss'></a>

# Step : $A_{\rm SS}^{\textrm{align}}$ \[Back to [top](#toc)\]
$$\label{aalignss}$$

The potential $A_{\rm SS}^{\rm align}$ is expressed in [RBB2023](https://arxiv.org/abs/2303.18046) Equation A6a:
\begin{equation*}
    A_{\rm SS}^{\rm align} = \frac{1}{r^4}\left[ \frac{9a_{+}^{2}}{8} - \frac{5}{4}\delta {a}_{-}{a}_{+} + a_{-}^{2}\left( \frac{\nu}{2} + \frac{1}{8} \right) \right] + \frac{1}{r^5}\left[ a_{+}^{2}\left( -\frac{175\nu}{64} - \frac{225}{64} \right) + \delta {a}_{-}{a}_{+}\left( \frac{117}{32} - \frac{39\nu}{16} \right) + a_{-}^{2} \left( \frac{21\nu^{2}}{16} - \frac{81\nu}{64} - \frac{9}{64} \right) \right].
\end{equation*}

Here, $r$ is the radial distance, $\nu$ is defined in [this cell](#nu) and ${a}_{\pm}$ is defined [in this cell](#a). 

In [12]:
%%writefile -a $Ccodesdir/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt
AalignSS = (
    (1/r**4)*(
        ap**2*sp.Rational(9,8)
        - delta*ap*am*sp.Rational(5,4)
        + am**2*(
            nu*sp.Rational(1,2) 
            + sp.Rational(1,8)
        )
    )
    + (1/r**5)*(
        ap**2*(
            -nu*sp.Rational(175,64) 
            - sp.Rational(225,64)
        )
        + delta*ap*am*(
            -nu*sp.Rational(39,16) 
            + sp.Rational(117,32)
        )
        + am**2*(
            nu*nu*sp.Rational(21,16) 
            - nu*sp.Rational(81,64) 
            - sp.Rational(9,64)
        )
    )
)

Appending to Hamiltonian/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt


<a id='bnpalignss'></a>

# Step : $B_{np,\rm SS}^{\textrm{align}}$ \[Back to [top](#toc)\]
$$\label{bnpalignss}$$

The potential $B_{\rm SS}^{\rm align}$ is expressed in [RBB2023](https://arxiv.org/abs/2303.18046) Equation A6b:
\begin{equation*}
    B_{np,\rm SS}^{\rm align} = \frac{1}{r^3}\left[ a_{+}^{2}\left(3\nu + \frac{45}{16}\right) - \frac{21}{8}\delta {a}_{+}{a}_{-} + a_{-}^{2}\left( \frac{3\nu}{4} - \frac{3}{16} \right) \right] + \frac{1}{r^4}\left[ a_{+}^{2}\left( -\frac{1171\nu}{64} - \frac{861}{64} \right) + \delta {a}_{+}{a}_{-}\left( \frac{449}{32} + \frac{13\nu}{16} \right) + a_{-}^{2} \left( \frac{\nu^{2}}{16} + \frac{115\nu}{64} - \frac{37}{64} \right) \right].
\end{equation*}

Here, $r$ is the radial distance, $\nu$ is defined in [this cell](#nu), and ${a}_{\pm}$ is defined [in this cell](#a).

In [13]:
%%writefile -a $Ccodesdir/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt
BnpalignSS = (
    (1/r**3)*( 
        ap**2*(
            3*nu 
            + sp.Rational(45,16)
        )
        - delta*ap*am*sp.Rational(21,8)
        + am**2*(
            nu*sp.Rational(3,4) 
            - sp.Rational(3,16)
        )
    )
    + (1/r**4)*( 
        ap**2*(
            -nu*sp.Rational(1171,64) 
            - sp.Rational(861,64)
        )
        + delta*ap*am*(
            nu*sp.Rational(13,16) 
            + sp.Rational(449,32)
        )
        + am**2*(
            nu*nu*sp.Rational(1,16) 
            + nu*sp.Rational(115,64) 
            - sp.Rational(37,64)
        ) 
    )
)

Appending to Hamiltonian/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt


<a id='qalignss'></a>

# Step : $Q_{\rm SS}^{\textrm{align}}$ \[Back to [top](#toc)\]
$$\label{qalignss}$$

The potential $Q_{\rm SS}^{\rm align}$ is expressed in [RBB2023](https://arxiv.org/abs/2303.18046) Equation A6c:
\begin{equation*}
    Q_{\rm SS}^{\rm align} = \frac{ p_{r}^{4} }{ r^3 }\left[ a_{+}^{2}\left(-5\nu^2 + \frac{165\nu}{32} + \frac{25}{32} \right) + \delta {a}_{+}{a}_{-}\left( \frac{45\nu}{8} - \frac{5}{16} \right) + a_{-}^{2} \left( -\frac{15\nu^{2}}{8} + \frac{75\nu}{32} - \frac{15}{32} \right) \right].
\end{equation*}

Here, $r$ is the radial distance, $p_{r}$ is defined in [this cell](#pr), $\nu$ is defined in [this cell](#nu), and ${a}_{\pm}$ is defined in [this cell](#a). 

In [14]:
%%writefile -a $Ccodesdir/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt
QalignSS = ((pr**4)/(r**3))*( 
        ap**2*( 
            -5*nu*nu 
            + nu*sp.Rational(165,32) 
            + sp.Rational(25,32)
        )
        + delta*ap*am*(
            nu*sp.Rational(45,8) 
            - sp.Rational(5,16)
        )
        + am**2*(
            -nu*nu*sp.Rational(15,8) 
            + nu*sp.Rational(75,32) 
            - sp.Rational(15,32)
        ) 
    )

Appending to Hamiltonian/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt


<a id='pr'></a>

# Step : The canonical momentum $p_{r}$ \[Back to [top](#toc)\]
$$\label{pr}$$

The canonical momentum is expressed in [KB2023](https://arxiv.org/abs/2303.18143) Equation 42:
\begin{equation*}
    p_r = \frac{p_{r_*}}{\xi(r)}.
\end{equation*}

Here, $p_{r_*}$ is the tortoise momentum, and $\xi(r)$ is defined in [this cell](#xi).

In [15]:
%%writefile -a $Ccodesdir/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt
pr = prstar/xi

Appending to Hamiltonian/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt


<a id='xi'></a>

# Step : The tortoise transformation $\xi(r)$ \[Back to [top](#toc)\]
$$\label{xi}$$

The canonical momentum is expressed in [KB2023](https://arxiv.org/abs/2303.18143) Equation 44:
\begin{equation*}
    \xi = \frac{\sqrt{D_{\rm noS}} \left( A_{\rm noS} + a_{+}^{2}/r^{2} \right) }{1 + a_{+}^2/r^2}.
\end{equation*}

Here, $r$ is the radial distance. The potentials: $D_{\rm noS}$ is defined in [this cell](#dnons), and $A_{\rm noS}$ is defined in [this cell](#anons). The spin combination $a_{+}$ is given in [this cell](#a)

In [16]:
%%writefile -a $Ccodesdir/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt
xi = sp.sqrt(Dnons) * ( Anons + ap**2*u*u ) / (1 + ap**2*u*u)

Appending to Hamiltonian/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt


# Step : The Non-Spinning Contributions

<a id='anons'></a>

# Step : $A_{\textrm{noS}}$ \[Back to [top](#toc)\]
$$\label{anons}$$

The Taylor potential $A_{\rm noS}$ is expressed in [KB2023](https://arxiv.org/abs/2303.18143) Equation 21:
\begin{equation*}
    {A}_{\rm noS}^{\rm Tay} = 1 + - 2u + 2\nu u^3 + \nu\left(\frac{94}{3} - \frac{41\pi^2}{32} \right)u^4 + \left[ \nu\left( \frac{2275\pi^2}{512} - \frac{4237}{60} + \frac{128\gamma_E}{5} + \frac{256 \ln 2}{5} \right) + \nu^2\left(\frac{41\pi^2}{32} - \frac{221}{6}\right) + \frac{64}{5}\nu \ln u \right]u^5 + \left[\nu a_6 - \nu\left(\frac{144\nu}{5} + \frac{7004}{105}\right)\ln u \right]u^6.
\end{equation*}

Here, $u$ is the inverse of the radial distance, defined in [this cell](#u). $\gamma_E$ is the Euler-Mascheroni constant, and $a_{6}$ is chosen as a free parameter that we will fit to NR simulations.

The non-spinning potential $A_{\rm noS}$ is defined as the Padé approximant:

\begin{equation*}
    A_{\rm noS} = P^{1}_{5}\left[ A_{\rm noS}^{\rm Tay} \right]
\end{equation*}

For consistency, we use the 

In [17]:
%%writefile -a $Ccodesdir/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt
Anons = 7680.0*r**4*(-5416406.59541186*nu**2 + 28.0*nu*(1920.0*a6 + 733955.307463037) + 2048.0*nu*(756.0*nu + 336.0*r + 407.0)*sp.log(r) - 7.0*r*(-185763.092693281*nu**2 + 938918.400156317*nu - 245760.0) - 3440640.0)/(241555486248.807*nu**4 + 1120.0*nu**3*(-17833256.898555*r**2 - 163683964.822551*r - 1188987459.03162) + 7.0*nu**2*(-39321600.0*a6*(3.0*r + 59.0) + 745857848.115604*a6 + 1426660551.8844*r**5 - 3089250703.76879*r**4 - 6178501407.53758*r**3 + 2064783811.32587*r**2 + 122635399361.987*r + 276057889687.011) + 67645734912.0*nu**2*sp.log(r)**2 + 53760.0*nu*(7680.0*a6*(r**4 + 2.0*r**3 + 4.0*r**2 + 8.0*r + 16.0) + 128.0*r*(-6852.34813868015*r**4 + 4264.6962773603*r**3 + 8529.39255472061*r**2 + 13218.7851094412*r - 33722.4297811176) + 113485.217444961*r*(-r**4 + 2.0*r**3 + 4.0*r**2 + 8.0*r + 16.0) + 148.04406601634*r*(349.0*r**4 + 1926.0*r**3 + 3852.0*r**2 + 7704.0*r + 36400.0)) + 32768.0*nu*(-1882456.23663972*nu**2 - 38842241.4769507*nu + 161280.0*r**5 + 480.0*r**4*(756.0*nu + 1079.0) + 960.0*r**3*(756.0*nu + 1079.0) + 1920.0*r**2*(588.0*nu + 1079.0) + 240.0*r*(-3024.0*nu**2 - 7466.27061066206*nu + 17264.0) + 13447680.0)*sp.log(r) + 13212057600.0*r**5)

Appending to Hamiltonian/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt


<a id='dnons'></a>

# Step : $\bar{D}_{\textrm{noS}}$ \[Back to [top](#toc)\]
$$\label{dnons}$$

The Taylor potential $\bar{D}_{\rm noS}$ is expressed in [KB2023](https://arxiv.org/abs/2303.18143) Equation 23:
\begin{equation*}
    \bar{D}_{\rm noS}^{\rm Tay} = 1 + 6\nu u^2 + \left(-6\nu^2 + 52\nu\right) u^3 + \left[ \nu\left( -\frac{23761\pi^2}{1536} - \frac{533}{45} + \frac{1184\gamma_E}{15} - \frac{6496 \ln 2}{15} + \frac{2916 \ln 3}{5}\right) + \nu^2\left(\frac{123\pi^2}{16} - 260\right) + \frac{592}{15}\nu \ln u \right]u^4 + \left[ \nu\left( \frac{294464}{175} - \frac{2840\gamma_E}{7} - \frac{63707\pi^2}{512} + \frac{120648 \ln 2}{35} - \frac{19683}{7} \right) \left( \frac{1069}{3} - \frac{205\pi^2}{16} \right)\nu^3 + \left( d^{\nu^2}_{5} - \frac{6784\gamma_E}{15} + \frac{67736}{105} + \frac{58320 \ln 3}{7} - \frac{326656 \ln 2}{21} \right)\nu^2 \right]u^5.
\end{equation*}

Here, $u$ is the inverse of the radial distance, defined in [this cell](#u). $\gamma_E$ is the Euler-Mascheroni constant, defined in [this cell](#emgamma) and $d^{\nu^2}_{5}$ is an NR calibration coefficient that is set to zero.

The non-spinning potential $\bar{D}_{\rm noS}$ is defined as the Padé approximant:

\begin{equation*}
    D_{\rm noS} = P^{2}_{3}\left[ D_{\rm noS}^{\rm Tay} \right]
\end{equation*}

In [18]:
%%writefile -a $Ccodesdir/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt
Dnons = r*(6730497718123.02*nu**3 + 22295347200.0*nu**2*d5 + 133772083200.0*nu**2*r**2 + 1822680546449.21*nu**2*r + 80059249540278.2*nu**2 + 22295347200.0*nu*d5*r - 193226342400.0*nu*d5 + 2589101062873.81*nu*r**2 + 10611661054566.2*nu*r - 12049908701745.2*nu + 5107745331375.71*r**2 - 326837426.241486*r*(14700.0*nu + 42911.0) - 39476764256925.6*r - (-5041721180160.0*nu**2 - 25392914995744.3*nu - 879923036160.0*r**2 - 283115520.0*r*(14700.0*nu + 42911.0) + 104186110149937.0)*sp.log(r) + 5787938193408.0*sp.log(r)**2 + 275059053208689.0)/(55296.0*nu*(14515200.0*nu**3 - 42636451.6032331*nu**2 - 7680.0*nu*(315.0*d5 + 890888.810272497) + 4331361844.61149*nu + 1002013764.01019) - 967680.0*r**3*(-138240.0*nu**2 - 2675575.66847905*nu - 5278341.3229329) - 9216.0*r**2*(-197773496.793534*nu**2 - 7680.0*nu*(315.0*d5 + 405152.309729121) + 2481453539.84635*nu + 5805304367.87913) + r*(5927865218923.02*nu**3 + 70778880.0*nu**2*(315.0*d5 + 2561145.80918574) - 138141470005001.0*nu**2 - 4718592.0*nu*(40950.0*d5 + 86207832.4415642) + 450172889755120.0*nu + 86618264430493.3*(1 - 0.496948781616935*nu)**2 + 188440788778196.0) + 5787938193408.0*r*sp.log(r)**2 + (-1698693120.0*nu*(11592.0*nu + 69847.0) + 879923036160.0*r**3 + 283115520.0*r**2*(14700.0*nu + 42911.0) + 49152.0*r*(102574080.0*nu**2 + 409207698.136075*nu - 2119671837.36038))*sp.log(r))
d5 = 0

Appending to Hamiltonian/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt


<a id='qnons'></a>

# Step : $Q_{\textrm{noS}}$ \[Back to [top](#toc)\]
$$\label{qnons}$$

The potential $Q_{\rm noS}$ is expressed in [PB2023](https://arxiv.org/pdf/2303.18039.pdf) Equation A2:

\begin{align}
\label{QpmTay}
Q_\text{noS} &= {p_{r_*}^4} \Bigg\lbrace 2 (4 - 3 \nu) \nu u^2 
+u^3 \left[10 \nu ^3-131 \nu ^2+\nu  \left(-\frac{4348}{15}+\frac{496256 \ln 2}{45}-\frac{33048 \ln 3}{5}\right)\right]
+ u^4 \Bigg[\left(792-\frac{615 \pi ^2}{32}\right) \nu ^3  \nonumber\\
&\qquad
+\nu ^2 \left(-\frac{592 \ln u}{5}+\frac{31633 \pi ^2}{512}-\frac{1184 \gamma_E}{5}+\frac{45683}{105}+\frac{33693536 \ln 2}{105}-\frac{6396489 \ln 3}{70}-\frac{9765625 \ln 5}{126}\right) \nonumber\\
&\qquad
+\nu  \left(\frac{5428 \ln u}{105}+\frac{1249177}{1050}-\frac{93031 \pi ^2}{1536}+\frac{10856 \gamma_E }{105}-\frac{4396376}{105} \ln 2+\frac{9765625 \ln 5}{504}-\frac{601911 \ln 3}{280}\right)\Bigg] 
+ \frac{88703 \pi  \nu  u^{9/2}}{1890} \Bigg\rbrace 
\nonumber\\
&\quad 
+ {p_{r_*}^6} \Bigg\lbrace
u^2 \left[6 \nu ^3-\frac{27 \nu ^2}{5}+\nu  \left(-\frac{827}{3}-\frac{1}{25} 2358912 \ln 2+\frac{1399437 \ln 3}{50}+\frac{390625 \ln 5}{18}\right)\right] \nonumber\\
&\qquad
+ u^3 \Bigg[-14 \nu ^4+188 \nu ^3+\nu ^2 \left(\frac{154229}{75}-\frac{4998308864 \ln 2}{1575}+\frac{26171875 \ln 5}{18}-\frac{45409167 \ln 3}{350}\right) \nonumber\\
&\quad\qquad
+\nu  \left(-\frac{860317}{1050}+\frac{305146624 \ln 2}{945}+\frac{35643726 \ln 3}{175}-\frac{52468750 \ln 5}{189}\right)\Bigg]
-\frac{2723471 \pi  \nu  u^{7/2}}{756000} \Bigg\rbrace 
\nonumber\\
&\quad
+ {p_{r_*}^8} \Bigg\lbrace
u \nu  \left[-\frac{35772}{175}+\frac{21668992 \ln 2}{45}+\frac{6591861 \ln 3}{350}-\frac{27734375 \ln 5}{126}\right]
+\frac{5994461 \pi  \nu  u^{5/2}}{12700800} + u^2 \Bigg[-6 \nu ^4+\frac{24 \nu ^3}{7} \nonumber\\
&\quad\qquad
+\nu ^2 \left(\frac{870976}{525}+\frac{703189497728 \ln 2}{33075}+\frac{332067403089 \ln 3}{39200}-\frac{13841287201 \ln 7}{4320}-\frac{468490234375 \ln 5}{42336}\right) \nonumber\\
&\quad\qquad
+\nu  \left(-\frac{2222547}{2450}-\frac{1347019456}{525}  \ln 2+\frac{278690984375 \ln 5}{169344}+\frac{13841287201 \ln 7}{17280}-\frac{346536085761 \ln 3}{156800}\right)\Bigg]
\Bigg\rbrace.
\end{align}

Here, $u$ is the inverse of the radial distance, defined in [this cell](#u).$a_{6}$ is an NR calibration coefficient defined in [this cell](#nr). Following the discussion under Equation 45 in [KB2023](https://arxiv.org/abs/2303.18143), we can replace the canonical momentum $p_r$ in the above equation with the tortoise momentum $p_{r_{*}}$, which is an input variable.  $\gamma_E$ is the Euler-Mascheroni constant, and is defined in the sympy library.

We note small disagreements with the pyseobnr code and the value of the coefficients in the literature. For consistency with pyseobnr we are using their version of Qnos instead of our version (saving our code here for future use):

```lang-py
Qnos = Qnosterm1 + Qnosterm2 + Qnosterm3
Qnosterm1 = (prstar**4) *(
    2 * (4 - 3 * nu) * nu * u**2
    + u**3 * (
        10 * nu**3 
        - 131 * nu**2 
        + nu * (
            -sp.Rational(4348, 15) + sp.Rational(496256,45) * sp.log(2) - sp.Rational(33048,5) * sp.log(3)
        )
    )
    + u**4 * (
        (792 - sp.Rational(615,32) * sp.pi**2) * nu**3
        + nu**2 * (
            - sp.Rational(592,5)*sp.log(u) + sp.Rational(31633,512) * sp.pi**2  - sp.Rational(1184,5) * sp.EulerGamma  
            + sp.Rational(45683, 105) + sp.Rational(33693536,105) * sp.log(2) - sp.Rational(6396489,70) * sp.log(3) 
            - sp.Rational(9765625,126) * sp.log(5)
        )
        + nu * (
            sp.Rational(5428,105)*sp.log(u) + sp.Rational(1249177, 1050) - sp.Rational(93031,1536) * sp.pi**2 
            + sp.Rational(10856,105) * sp.EulerGamma - sp.Rational(4396376, 105) * sp.log(2) + sp.Rational(9765625,504) * sp.log(5)
            - sp.Rational(601911,280) * sp.log(3)
        )
    )
    + sp.Rational(88703, 1890) * sp.pi * nu * u**(sp.Rational(9, 2))
)

Qnosterm2 = (prstar**6) * ( 
    u**2 * ( 6*nu**3 
            - sp.Rational(27,5)*nu**2
            + nu*(
                -sp.Rational(827,3) - sp.Rational(2358912,25)*sp.log(2) + sp.Rational(1399437,50)*sp.log(3) 
                + sp.Rational(390625,18)*sp.log(5)
            ) 
           )
    + u**3 * ( -14*nu**4 
              + 188*nu**3
              + nu**2 * ( 
                  sp.Rational(154229,75) - sp.Rational(4998308864,1575)*sp.log(2) + sp.Rational(26171875,18)*sp.log(5)
                  - sp.Rational(45409167,350)*sp.log(3)
              )
              + nu * ( 
                  -sp.Rational(860317,1050) + sp.Rational(305146624,945)*sp.log(2) + sp.Rational(35643726,175)*sp.log(3)
                  - sp.Rational(52468750,189)*sp.log(5)
              )
             )
    - sp.Rational(2723471,756000)*sp.pi*nu*u**(sp.Rational(7,2))
)

Qnosterm3 = (prstar**8) * ( 
    u*nu * ( 
        -sp.Rational(35772,175) + sp.Rational(21668992,45)*sp.log(2) + sp.Rational(6591861,350)*sp.log(3) 
        - sp.Rational(27734375,126)*sp.log(5)
    )
    +u**2 * (
        -6*nu**4 + 
        sp.Rational(24,7)*nu**3
        +nu**2 * (
            sp.Rational(870976,525) + sp.Rational(703189497728,33075)*sp.log(2) + sp.Rational(332067403089,39200)*sp.log(3)
            -sp.Rational(13841287201,4320)*sp.log(7) - sp.Rational(468490234375,42336)*sp.log(5)
        )
        +nu * (
            -sp.Rational(2222547,2450) - sp.Rational(1347019456,525)*sp.log(2) + sp.Rational(278690984375,169344)*sp.log(5)
            +sp.Rational(13841287201,17280)*sp.log(7) - sp.Rational(346536085761,156800)*sp.log(3)
        )
    )
    +sp.Rational(5994461,12700800)*sp.pi*nu*u**(sp.Rational(5,2))
) 
```

In [19]:
%%writefile -a $Ccodesdir/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt
Qnos = (0.121954868780449*nu*prstar**8/r + prstar**6*(6.0*nu**3 - 5.4*nu**2 - 2.78300763695006*nu)/r**2 + prstar**4*(10.0*nu**3 - 131.0*nu**2 + 92.7110442849544*nu)/r**3) + (prstar**8*(-6.0*nu**4 + 3.42857142857143*nu**3 + 3.33842023648322*nu**2 + 1.38977750996128*nu)/r**2 + prstar**6*(-14.0*nu**4 + 188.0*nu**3 - 89.5298327361234*nu**2 - 33.9782122170436*nu)/r**3 + prstar**4*(602.318540416564*nu**3 + nu**2*(118.4*sp.log(r) - 1796.13660498019) + nu*(452.542166996693 - 51.6952380952381*sp.log(r) ))/r**4) + (1.48275342024365*nu*prstar**8/r**2.5 - 11.3175085791863*nu*prstar**6/r**3.5 + 147.443752990146*nu*prstar**4/r**4.5) + prstar**4*(-6.0*nu**2 + 8.0*nu)/r**2

Appending to Hamiltonian/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt


<a id='a'></a>

# Step : The spin combinations ${a}_{\pm}$ \[Back to [top](#toc)\]
$$\label{a}$$

In this section, we define the spin combinations as expressed in [RBB2023](https://arxiv.org/abs/2303.18046) Equation 3:

\begin{equation*}
    {a}_{\pm} = \frac{m_1}{M}{\chi}_{1} \pm \frac{m_2}{M}{\chi}_{2}.
\end{equation*}

Where, ${\chi}_{1,2}$ are the dimensionless spins. These are also provided as input parameters to the Hamiltonian function.

In [20]:
%%writefile -a $Ccodesdir/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt
ap = (m1*chi1 + m2*chi2)/M
am = (m1*chi1 - m2*chi2)/M

Appending to Hamiltonian/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt


# Step: Calibration Coefficients and Constants

<a id='ga'></a>

# Step : The gyro-gravitomagnetic factors $g_{a_\pm}$ \[Back to [top](#toc)\]
$$\label{ga}$$

In this section, we define the gyro-gravitomagnetic factors as expressed in [KB2023](https://arxiv.org/abs/2303.18143) Equation 28:

\begin{equation*}
\begin{aligned}
&\begin{aligned}
g_{a_{+}}^{3.5 \mathrm{PN}}=\frac{7}{4} & +\left[\frac{p_{\phi}^2}{r^2}\left(-\frac{45 \nu}{32}-\frac{15}{32}\right)+\frac{1}{r}\left(\frac{23 \nu}{32}-\frac{3}{32}\right)\right] \\
+ & {\left[\frac{p_{\phi}^4}{r^4}\left(\frac{345 \nu^2}{256}+\frac{75 \nu}{128}+\frac{105}{256}\right)\right.} \\
& +\frac{p_{\phi}^2}{r^3}\left(-\frac{1591 \nu^2}{768}-\frac{267 \nu}{128}+\frac{59}{256}\right) \\
& \left.+\frac{1}{r^2}\left(\frac{109 \nu^2}{192}-\frac{177 \nu}{32}-\frac{5}{64}\right)\right],
\end{aligned}\\
&\begin{aligned}
g_{a_{-}}^{3.5 \mathrm{PN}}= & \frac{1}{4}+\left[\frac{p_{\phi}^2}{r^2}\left(\frac{15}{32}-\frac{9 \nu}{32}\right)+\frac{1}{r}\left(\frac{11 \nu}{32}+\frac{3}{32}\right)\right] \\
+ & {\left[\frac{p_{\phi}^4}{r^4}\left(\frac{75 \nu^2}{256}-\frac{45 \nu}{128}-\frac{105}{256}\right)\right.} \\
& +\frac{p_{\phi}^2}{r^3}\left(-\frac{613 \nu^2}{768}-\frac{35 \nu}{128}-\frac{59}{256}\right) \\
& \left.+\frac{1}{r^2}\left(\frac{103 \nu^2}{192}-\frac{\nu}{32}+\frac{5}{64}\right)\right].
\end{aligned}
\end{aligned}
\end{equation*}
Where, ${\bf \chi}_{1,2}$ are the dimensionless spin vectors. These are also provided as input parameters to the Hamiltonian function. More importantly, we express the magnitude of the combinations and their dot product.

In [21]:
%%writefile -a $Ccodesdir/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt
gap = (
        sp.Rational(7, 4)
        + (pphi**2 / r**2) * (
            -sp.Rational(45, 32) * nu - sp.Rational(15, 32)
        )
        + (1 / r) * (
            sp.Rational(23, 32) * nu - sp.Rational(3, 32)
        )
        + (pphi**4 / r**4) * (
            sp.Rational(345, 256) * nu**2 + sp.Rational(75, 128) * nu + sp.Rational(105, 256)
        )
        + (pphi**2 / r**3) * (
            -sp.Rational(1591, 768) * nu**2 - sp.Rational(267, 128) * nu + sp.Rational(59, 256)
        )
        + (1 / r**2) * (
            sp.Rational(109, 192) * nu**2 - sp.Rational(177, 32) * nu - sp.Rational(5, 64)
        )
    )
gam = (
        sp.Rational(1, 4)
        + (pphi**2 / r**2) * (
            sp.Rational(15, 32) - sp.Rational(9, 32) * nu
        )
        + (1 / r) * (
            sp.Rational(11, 32) * nu + sp.Rational(3, 32)
        )
        + (pphi**4 / r**4) * (
            sp.Rational(75, 256) * nu**2 - sp.Rational(45, 128) * nu - sp.Rational(105, 256)
        )
        + (pphi**2 / r**3) * (
            -sp.Rational(613, 768) * nu**2 - sp.Rational(35, 128) * nu - sp.Rational(59, 256)
        )
        + (1 / r**2) * (
            sp.Rational(103, 192) * nu**2 - sp.Rational(1, 32) * nu + sp.Rational(5, 64)
        )
    )

Appending to Hamiltonian/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt


<a id='a6'></a>

# Step :  $a_{6}$ \[Back to [top](#toc)\]
$$\label{a6}$$

In this section, we define the calibration coefficient $a_6$ as documented [here](https://dcc.ligo.org/public/0186/T2300060/002/SEOBNRv5HM.pdf):
\begin{equation*}
a_6 = 41.7877875 - 3021.93382 \nu + 33414.4394 \nu^2 - 169019.14 \nu^3 + 329523.262 \nu^4
\end{equation*}

Where, $\nu$ is the symmetric mass ratio and is defined in [this cell](#nu).

In [22]:
%%writefile -a $Ccodesdir/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt
a6 = 41.7877875 - 3021.93382 * nu + 33414.4394 * nu**2 - 169019.14 * nu**3 + 329523.262 * nu**4

Appending to Hamiltonian/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt


# Step : The Mass Combinations

<a id='nu'></a>

# Step :  The symmetric mass ratio $\nu$ \[Back to [top](#toc)\]
$$\label{nu}$$

In this section, we define the symmetric mass ratio $\nu$ as expressed in [RBB2023](https://arxiv.org/abs/2303.18046) Equation 1:
\begin{equation*}
\nu = \frac{m_1 m_2}{M^2}
\end{equation*}

Where, $M$ is the total mass defined in [this cell](#m).

In [23]:
%%writefile -a $Ccodesdir/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt
nu = m1*m2/(M**2)

Appending to Hamiltonian/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt


<a id='delta'></a>

# Step :  $\delta$ \[Back to [top](#toc)\]
$$\label{delta}$$

In this section, we define $\delta$ as expressed in [RBB2023](https://arxiv.org/abs/2303.18046) Equation 1:
\begin{equation*}
\delta = \frac{m_1 - m_2}{M}
\end{equation*}

Where, $M$ is the total mass defined in [this cell](#m).

In [24]:
%%writefile -a $Ccodesdir/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt
delta = (m1-m2)/M

Appending to Hamiltonian/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt


<a id='m'></a>

# Step :  The total mass $M$ \[Back to [top](#toc)\]
$$\label{m}$$

In this section, we define the total mass as expressed in [RBB2023](https://arxiv.org/abs/2303.18046) Equation 1:
\begin{equation*}
M = m_1 + m_2
\end{equation*}

Where, $m_{1,2}$ are the individual masses of the black holes, given as input parameters.

In [25]:
%%writefile -a $Ccodesdir/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt
M = m1+m2

Appending to Hamiltonian/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt


<a id='u'></a>

# Step :  The inverse radius $u$ \[Back to [top](#toc)\]
$$\label{u}$$

In this section, we define the total mass as expressed in [RBB2023](https://arxiv.org/abs/2303.18046) Equation 1:
\begin{equation*}
u = \frac{1}{r}
\end{equation*}

Where, $r$ is the radial distance.

In [26]:
%%writefile -a $Ccodesdir/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt
u = 1/r

Appending to Hamiltonian/v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt


# Step : Saving the expressions

Up till now, the expressions required for the Hamiltonian have been stored in a .txt file. for the sake of readability, some of the expressions have been written in more than one line. 

Therefore, we save the expressions in one-line format to parse into sympy for generating optimized code or derivative expressions.

In [27]:
import os 
with open(os.path.join(Ccodesdir,"v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top-oneline.txt"), "w") as output:
    count = 0
    # Read output of this notebook
    for line in list(open(os.path.join(Ccodesdir,"v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt"),"r")):
        # Read the first line
        if count == 0:
            prevline=line
        #Check if prevline is a complete expression
        elif "=" in prevline and "=" in line:
            output.write("%s\n" % prevline.strip('\n'))
            prevline=line
        # Check if line needs to be adjoined to prevline
        elif "=" in prevline and not "=" in line:
            prevline = prevline.strip('\n')
            prevline = (prevline+line).replace(" ","")
        # Be sure to print the last line.
        if count == len(list(open(os.path.join(Ccodesdir,"v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top.txt"))))-1:
            if not "=" in line:
                print("ERROR. Algorithm not robust if there is no equals sign on the final line. Sorry.")
                sys.exit(1)
            else:
                output.write("%s" % line)
        count = count + 1

with open(os.path.join(Ccodesdir,"v5HM_BOB_Hamiltonian-Calibration-Hreal_on_bottom.txt"), "w") as output:
    for line in reversed(list(open(os.path.join(Ccodesdir,"v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top-oneline.txt"),"r"))):
        output.write("%s\n"%line.rstrip())


# Generate unoptimized python code

In this step, we'll simply store the Hamiltonian expressions in a python module as is in order to run preliminary validation tests. In addition, we will compare the results of our Hamiltonian function with that of the pyseobnr Hamiltonian function.

In [28]:
with open(os.path.join(Ccodesdir,"v5HM_BOB_unoptimized_hamiltonian-calibration.py"), "w") as output:
    output.write("import numpy as np\ndef v5HM_BOB_unoptimized_hamiltonian-calibration(m1, m2, r, prstar, pphi, chi1, chi2,a6,dSO,verbose = False):\n")
    for line in reversed(list(open(os.path.join(Ccodesdir,"v5HM_BOB_Hamiltonian-Calibration-Hreal_on_top-oneline.txt"),"r"))):
        output.write("    %s\n" % line.rstrip().replace("sp.sqrt", "np.sqrt").replace("sp.Rational",
                                "np.divide").replace("sp.log","np.log").replace("sp.pi","np.pi").replace("sp.EulerGamma","np.euler_gamma"))
    output.write("    if not verbose:\n        return Hreal,xi\n    else:\n        return Hreal,xi,Aalign,Balignnp,Bkerreqnp,Qalign,Heven,Hodd,QalignSS,Qnos,Galigna3,gam,gap,SOcalib,u,nu,ap,am,r,prstar,pphi,chi1,chi2,m1,m2")
    
import Hamiltonian.v5HM_unoptimized_hamiltonian as Hreal_unopt
import Hamiltonian.pyseobnr_hamiltonian as Hreal_true
import numpy as np

#generate random input data
gt_pert_total = []
gt_pert_O1 = []
gt_pert_O2 = []
gt_pert_gtO3 = []
N = 10000
rng = np.random.default_rng(seed = 50)
nu = rng.random(N)*.25
chi1 = 2.*rng.random(N)-1.
chi2 = 2.*rng.random(N)-1.
m2 = (1 - np.sqrt(1 - 4*nu))*.5
m1 = (1 + np.sqrt(1 - 4*nu))*.5
r = rng.random(N)*17 + 3.
phi = rng.random(N)*2.*np.pi
prstar = rng.random(N)*20. - 10.
pphi = rng.random(N)*20. - 10.

# generate randomized perturbation in order to find the appropriate tolerance
pert_exponent = 1e-14;
pert_sign = 2*rng.integers(0,high = 2,size = N) - 1;
pert_mantissa = 3.*rng.random(N) + 1. ;
pert = 1. + pert_sign*pert_mantissa*pert_exponent
chi1pert = chi1*pert 
chi2pert = chi2*pert
m2pert = m2*pert
m1pert = m1*pert
rpert = r*pert
phipert = phi*pert
prstarpert = prstar*pert
pphipert = pphi*pert

nans = []
def E_rel(a,b):
    return np.abs((a - b)/a)

for i in range(N):
    q  = [r[i],phi[i]]
    p = [prstar[i],pphi[i]]
    qpert  = [rpert[i],phipert[i]]
    ppert = [prstarpert[i],pphipert[i]]
    H_unopt,xi = Hreal_unopt.v5HM_unoptimized_hamiltonian(m1[i],m2[i],r[i],prstar[i],pphi[i],chi1[i],chi2[i],verbose = False)
    H_pyseobnr,xi = Hreal_true.evaluate_H(q,p, chi1[i], chi2[i], m1[i], m2[i],verbose = False)
    H_pert,xi = Hreal_true.evaluate_H(qpert,ppert, chi1pert[i], chi2pert[i], m1pert[i], m2pert[i],verbose = False)
    e_rel = E_rel(H_pyseobnr,H_unopt)
    tol = E_rel(H_pyseobnr,H_pert)
    if (np.isnan(H_pyseobnr) and np.isnan(H_unopt) and np.isnan(H_pert)):
        nans.append(i)
    else:
        if e_rel>tol:
            gt_pert_total.append(i)
            if e_rel > 1000*tol:
                gt_pert_gtO3.append(i)
            elif e_rel > 100*tol:
                gt_pert_O2.append(i)
            elif e_rel > 10*tol:
                gt_pert_O1.append(i)

print("Of total ",str(N)," comparisons,\n",
     "number of cases with relative error (total)  greater than allowed: ",len(gt_pert_total),"\n",
     "number of cases with relative error O(10)    greater than allowed: ",len(gt_pert_O1),"\n",
     "number of cases with relative error O(100)   greater than allowed: ",len(gt_pert_O2),"\n",
     "number of cases with relative error O(1000+) greater than allowed: ",len(gt_pert_gtO3))



Of total  10000  comparisons,
 number of cases with relative error (total)  greater than allowed:  0 
 number of cases with relative error O(10)    greater than allowed:  0 
 number of cases with relative error O(100)   greater than allowed:  0 
 number of cases with relative error O(1000+) greater than allowed:  0
